# Making HTTP Requests

In Python, there are many libraries to make HTTP requests. We will use a 3rd-party library called "requests", which is very easy to use. 

Making a "GET" request is as simple as: 

```python
import requests

res = requests.get(url) # returns a "Response" object
res.content # has the "body" of the response
```

You might need to install the requests library! 

You can do that with the following code in a Jupyter cell: 

```python
! pip install requests
```

Or, if you're using anaconda, optionally you can also do: 

```python
! conda install -c anaconda requests
```

## Parsing JSON data

To parse JSON data in Python, we will use the "json" module: 

```python
import json
```

Read more about the module on the [documentation page](https://docs.python.org/3/library/json.html)!

All we care about for this part is the method "loads", which turns JSON data into a Python object: 

```python
json.loads(my_string_encoded_json)
```

## Pokemon API

There is a simple, open API called "pokeapi" that allows us to make requests and see how to use APIs. Like everything, we first look at the documentation: 

https://pokeapi.co/docs/v2.html

In [ ]:
# Let's see how to make a get request to the API: 
import requests
import json

res = requests.get('https://pokeapi.co/api/v2/pokemon?offset=20&limit=20')
json.loads(res.content)

In [ ]:
# Challenge: 
# Create a Dataframe with all the Pokemon names and their URLs. 

def get_pokes(url):
    # Make the HTTP request to the given url.
    content = requests.get(url).content
    # Parse the response as json
    d = json.loads(content)
    return d['next'],d['results']
    # return the "next" and the "results" (as a 2-tuple!)


def catch_em_all(url):
    pokes = []
    
    # While loop! Like a for-loop, 
    # but goes on for an indetermined amount
    # of time:
    while url:
        url, results = get_pokes(url)
        pokes += results
    return pokes
        
    
list_of_pokes = catch_em_all('https://pokeapi.co/api/v2/pokemon')

# This data is most naturally represented as a list of dictionaries. 
# How can we create a dataframe from a list of dictionaries? 
# Try to find out on your own, from the internet!

# TODO: turn list_of_pokes into a dataframe.

In [ ]:
len(list_of_pokes)

## Project: Live Exchange Rates

Imagine that you work with financial assets which are denominated in different currencies. You analyze this data regularly, and want to create a "transformation" function that transforms all your assets into EUR prices, based on today's exchange rate. 

Your data with the local-currency-denominated value of each asset lives in a file called "assets.csv" which should be located in the same folder as this notebook. 

Write a "data loading" function that: 

1. Reads the data, given the path to the file. 
2. Returns a dataframe with an additional column that has the assets value in euros, as of today.

Use this free API to get today's exchange rates: https://exchangeratesapi.io/. You will need to read the documentation and try it out to see how it works. 

HINT: Write a separate function to get the current exchange rates! That can be reused!

In [2]:
import requests
import json
import pandas as pd

#read the assets file
assets_file = 'assets.csv'
fx_rates = 'https://api.exchangeratesapi.io/latest'

def get_rates():
    return json.loads(requests.get(fx_rates).content)['rates']

def create_dataframe(csv):
    assets = pd.read_csv(csv)
    assets["EUR_value"]=""
    rates_table = get_rates()
    assets['EUR_value'] = assets['value'] / assets['curr'].map(rates_table)
    return assets

create_dataframe(assets_file)

,value,curr,EUR_value
0,48.910052,THB,1.452716
1,16.505115,THB,0.490232
2,30.370579,INR,0.383891
3,14.126967,SEK,1.314161
4,23.406904,HKD,2.678105
5,24.869579,MYR,5.336018
6,26.431815,THB,0.785072
7,44.698794,PLN,10.497357
8,34.306495,KRW,0.026331
9,14.711549,BGN,7.522011
